In [18]:
from azureml.core import Experiment, Workspace
import pandas as pd
ws = Workspace.from_config()
experiment = Experiment(name="hello-world", workspace=ws)
run = experiment.start_logging()
df = pd.read_csv("winequality-white.csv", delimiter=";")
run.log("nrows", df.shape[0])
run.log_table("data dimensions", {"rows":df.shape[0], 
               "columns": df.shape[1]})

run.complete()


In [19]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

# submitting script as an experiment

In [20]:
import os
os.makedirs("sample_script_experiment", exist_ok=True)

In [43]:
%%writefile sample_script_experiment/script.py
from azureml.core import Run
import pandas as pd

run = Run.get_context()
df = pd.read_csv("winequality-white.csv", delimiter=";")
run.log("observation_count", df.shape[0])
run.complete()

Overwriting sample_script_experiment/script.py


In [44]:
from azureml.core import ScriptRunConfig, Experiment
from azureml.core import Environment
env_config = CondaDependencies.create(pip_packages=["pandas", "azureml-core"], 
                         python_version="3.8")
env_config.save_to_file("sample_script_experiment", "env_config.yaml")
env = Environment.from_conda_specification(name="sample_script_env", file_path="sample_script_experiment/env_config.yaml")
experiment = Experiment(workspace=ws, name="script-sample-experiment")
src = ScriptRunConfig(source_directory="sample_script_experiment",
                     script="script.py",
                     environment=env)
run = experiment.submit(config=src)
run.wait_for_completion()

{'runId': 'script-sample-experiment_1643021986_e5b5c4d6',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2022-01-24T10:59:49.961627Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '118c092d-eade-4ff5-948f-7c121419cc3b'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'script.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': 'sample_script_env',
   'version': 'Autosave_2022-01-24T10:45:11Z_75113959',
   'python': {'interpreterPath': 'python',
    'userManagedDependencies': False,
    'condaDependencies

In [45]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…